In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import json
import pandas as pd
import numpy as np
import pyvista as pv
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lsh import LSH, L1_LSH

import time
import math
from scipy.spatial import KDTree

## Memory footprint

In [ ]:
def compute_memory_footprint(root, obj_ext):
    total_bytes = 0
    fpaths = [os.path.join(root, f) for f in filter(lambda x: x.endswith(obj_ext), os.listdir(root))]
    for path in fpaths:
        total_bytes += os.path.getsize(path)

    print(f"{total_bytes:,d} b")
    
compute_memory_footprint("/mnt/arteml/data/meshsdf/cars/meshes/", ".obj")
compute_memory_footprint("/mnt/arteml/classes/6850/data/trained_cars/", ".h5")
compute_memory_footprint("/mnt/arteml/classes/6850/thirdparty/DeepSDF/experiments/cars/ModelParameters/", "latest.pth")

# 3D meshes hashing

In [ ]:
def display_found_cars(search_results, scale=1.15, dpi=1000, save_suffix=""):
    renders_dir = "/mnt/arteml/data/meshsdf/cars/renders/"

    n_cols = max([len(sr) for sr in search_results])
    fig = plt.figure(dpi=dpi)
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(len(search_results), n_cols),
                     axes_pad=0.0,  # pad between axes in inch.
                 )

    for j_idx, search_line in enumerate(search_results):
        for i_idx, result in enumerate(search_line):
            name = result[1]["model_id"]
            img = plt.imread(os.path.join(renders_dir, name, "02.png"))

            shift_x = max(int( (img.shape[0] - img.shape[0] / scale) / 2), 1)
            shift_y = max(int( (img.shape[1] - img.shape[1] / scale) / 2), 1)

            img_zoomed_in = img[shift_x:-shift_x, shift_y:-shift_y, :]

            grid_cell = grid[j_idx * n_cols + i_idx]
            grid_cell.imshow(img_zoomed_in)
            grid_cell.set_xticks([])
            grid_cell.set_yticks([])

            grid_cell.spines['top'].set_visible(False)
            grid_cell.spines['right'].set_visible(False)
            grid_cell.spines['bottom'].set_visible(False)
            grid_cell.spines['left'].set_visible(False)

            if i_idx == 0:
                print_text = f"query\n{name}"
            else:
                print_text = f"dist: {result[2]:.2f}\n{name}"
            grid_cell.set_title(print_text, fontsize=1.5, y=-0.15)

    plt.plot()
    # plt.savefig(os.path.join(exp_dir, "comp" + save_suffix + ".png"), dpi=dpi)

### WeightEncoded NIRs

### Autoencoder NIRs

In [ ]:
import torch

import sys
sys.path.insert(1, "thirdparty/DeepSDF/")
from networks.deep_sdf_decoder import Decoder
import deep_sdf

In [ ]:
experiment_dir = "thirdparty/DeepSDF/experiments/cars/"

specs = json.load(open(os.path.join(experiment_dir, "specs.json")))
train_mapping = json.load(open(specs["TrainSplit"]))
test_mapping = json.load(open(specs["TestSplit"]))

# Load the model
decoder = torch.nn.DataParallel(Decoder(specs["CodeLength"],  **specs["NetworkSpecs"]))
saved_model_state = torch.load(
    os.path.join(experiment_dir, "ModelParameters", "latest.pth")
)
decoder.load_state_dict(saved_model_state["model_state_dict"])
decoder = decoder.module
decoder = decoder.eval()

# Load latent codes
latents = torch.load(os.path.join(experiment_dir, "LatentCodes/latest.pth"))["latent_codes"]["weight"].cpu().numpy()

In [ ]:
table = LSH(error_rate=0.05, max_k=30)

table.build(latents, objects=train_mapping)
table.plot_info()

In [ ]:
agg_results = []

for idx in tqdm([0, 1, 2, 3, 4, 5]):
    agg_results.append(table.get_n_nearest_neigbours(latents[idx], 8))

display_found_cars(agg_results)

### Plain meshes

In [ ]:
def spin_image(verts, res=(16, 16) ):
    # Normalise model
    verts -= verts.mean(axis=0)
    verts /= verts.std(axis=0)

    img = np.zeros(res, dtype=int)

    radiuses = np.sqrt( np.sum(verts[:, :2] ** 2, axis=1) )
    heights = verts[:, 2]
    
    h_idx = np.clip(np.round((1.5 + heights) / 3 * res[0]), 0, res[0] - 1).astype(int)
    r_idx = np.clip(np.round(radiuses / 2.5 * res[1]), 0, res[1] - 1).astype(int)
    
    for i, j in zip(h_idx, r_idx):
        img[i, j] += 1
    
    return img.reshape(-1).astype(float) / verts.shape[0]

In [ ]:
data_source = "/mnt/arteml/data/meshsdf/cars/meshes"
files = [x["model_id"] + ".obj" for x in train_mapping] #list(filter(lambda x: x.endswith(".obj"), os.listdir(data_source)))

features = []
objects  = []
for fname in tqdm(files):
    verts = np.array(pv.read(os.path.join(data_source, fname)).points)

    features.append(spin_image(verts))
    objects.append({"model_id": fname[:-4]})
features = np.array(features)

In [ ]:
table = LSH(error_rate=0.05, max_k=30)

table.build(features, objects=objects)
table.plot_info()

In [ ]:
agg_results = []

for idx in tqdm([0, 1, 2, 3, 4, 5]):
    agg_results.append(table.get_n_nearest_neigbours(features[idx], 8))

display_found_cars(agg_results)

In [ ]:
def display_found_cars(search_results, scale=1.15, dpi=1000, save_suffix=""):
    renders_dir = "/mnt/arteml/data/meshsdf/cars/renders/"

    n_cols = max([len(sr) for sr in search_results])
    fig = plt.figure(dpi=dpi)
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(len(search_results), n_cols),
                     axes_pad=0.0,  # pad between axes in inch.
                 )

    for j_idx, search_line in enumerate(search_results):
        for i_idx, result in enumerate(search_line):
            name = result[1]["model_id"]
            img = plt.imread(os.path.join(renders_dir, name, "02.png"))

            shift_x = max(int( (img.shape[0] - img.shape[0] / scale) / 2), 1)
            shift_y = max(int( (img.shape[1] - img.shape[1] / scale) / 2), 1)

            img_zoomed_in = img[shift_x:-shift_x, shift_y:-shift_y, :]

            grid_cell = grid[j_idx * n_cols + i_idx]
            grid_cell.imshow(img_zoomed_in)
            grid_cell.set_xticks([])
            grid_cell.set_yticks([])

            grid_cell.spines['top'].set_visible(False)
            grid_cell.spines['right'].set_visible(False)
            grid_cell.spines['bottom'].set_visible(False)
            grid_cell.spines['left'].set_visible(False)

            if i_idx == 0:
                print_text = f"query\n{name}"
            else:
                print_text = f"dist: {result[2]:.2f}\n{name}"
            grid_cell.set_title(print_text, fontsize=1.5, y=-0.15)

    plt.plot()
    # plt.savefig(os.path.join(exp_dir, "comp" + save_suffix + ".png"), dpi=dpi)

In [ ]:
import torch

import sys
sys.path.insert(1, "thirdparty/DeepSDF/")
from networks.deep_sdf_decoder import Decoder
import deep_sdf

In [ ]:
fpaths

['/mnt/arteml/data/meshsdf/cars/meshes/4ceeed48d1a48d4ce09e4fc69d1a2697.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/2076be423fa5bb9f9e908ecb03b3d6fb.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/a7065cc33270e551a3049d0dcf503cdf.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/b50f9931670e25ef44ccce632b473b8c.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/2c3a5d170774da5ae4a44d0583e1bb01.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/b659b096b9cd0de093532d4a06abe81.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/7e3349b2d3e833b0364d334d86d85d.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/84ffded0ab4b499588ed7b6315c3b4a.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/18d9ac72442260e0e97c6bb243ef67d.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/31055873d40dc262c7477eb29831a699.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/e05680db6f028c15bda733a39f84326d.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/936eb7eec6b76e4b421c2195de5e56c4.obj',
 '/mnt/arteml/data/meshsdf/cars/meshes/7825218160564f137039b7b9eba2e0